# Hypothesentests

**Zielsetzung und Vorgehen**

Die explorative Analyse hat klare Muster erkennen lassen.  
In diesem Abschnitt werden diese Muster systematisch geprüft, um zu entscheiden, welche Effekte tatsächlich als **Risikotreiber** gelten können und damit später als Features in das Risikomodell einfließen.

Die Tests orientieren sich bewusst an der späteren Modelllogik:  
Unfälle werden als Zähldaten behandelt und über einen log-Offset durch die jeweilige **Exposure** (Anzahl der Fahrten) normalisiert.  
Auf diese Weise wird direkt die **Unfallrate pro Fahrt** modelliert und die Ergebnisse sind unabhängig davon, wie stark bestimmte Stunden, Saisons oder Bike-Typen genutzt werden.

Für die weitere Modellierung ist es ausreichend, die drei wichtigsten Hypothesen aus der EDA zu prüfen – jene Effekte, die in der Praxis am stärksten mit dem Risiko verbunden sind:

1. **Safety in Numbers:** Zusammenhang zwischen Nutzungsintensität und Unfallrate.  
2. **Zeitliche Risikostruktur:** Einfluss von Stunde und Saison.  
3. **Bike-Typ:** Unterschiedliche Unfallraten von klassischen Fahrrädern und E-Bikes unter vergleichbaren Zeitbedingungen.

Die Hypothesentests bilden damit die Verbindung zwischen EDA und Feature Engineering:  
Sie zeigen, welche Muster stabil sind und sich für ein transparentes Risikomodell eignen.


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

import sys
sys.path.append("../..")

from utils.paths import DATA_SPATIAL
from utils.paths import DATA_TEMPORAL
from utils.paths import DATA_MODEL
from utils.model_summary import custom_summary

## Setup und Laden der Daten

In [26]:
path_spatial_master = os.path.join(DATA_SPATIAL, 'spatial_master.parquet')
df_spatial_master = pd.read_parquet(path_spatial_master)

In [27]:
path_cb_spatial = os.path.join(DATA_SPATIAL, 'citibike_spatial_granular.parquet')
df_cb_spatial = pd.read_parquet(path_cb_spatial)

In [28]:
path_cb_temporal = os.path.join(DATA_TEMPORAL, 'citibike_temporal.parquet')
df_cb_temporal = pd.read_parquet(path_cb_temporal)

In [29]:
path_nypd_spatial = os.path.join(DATA_SPATIAL, 'nypd_spatial_granular.parquet')
df_nypd_spatial = pd.read_parquet(path_nypd_spatial)

In [30]:
path_nypd_temporal = os.path.join(DATA_TEMPORAL, 'nypd_temporal.parquet')
df_nypd_temporal = pd.read_parquet(path_nypd_temporal)

## 1. H1 – Zusammenhang zwischen Nutzungsintensität und Unfallrate (Safety in Numbers)

**1. Motivation**  
In der räumlichen EDA zeigte sich, dass Zellen mit hoher Nutzungsintensität tendenziell eine niedrigere beobachtete Unfallrate aufweisen.  
Der Test soll klären, ob dieser Zusammenhang systematisch ist oder nur durch zufällige Schwankungen in wenig genutzten Zellen entsteht.

**2. Formale Hypothese**  
- $H_0: \text{Die Unfallrate pro Fahrt ist unabhängig vom Exposure-Decile.}$  
- $H_1: \text{Die Unfallrate pro Fahrt nimmt mit höherem Exposure-Decile ab.}$

**3. Datenbasis und Aggregation**  
Für jede Rasterzelle wird die Gesamt-Exposure (Anzahl der Fahrten) verwendet.  
Zur Glättung und zur besseren Vergleichbarkeit wird die Exposure in zehn gleich große Quantilsgruppen (Deciles) eingeteilt.  
Unfälle beziehen sich hier ausschließlich auf Fahrradbeteiligte (crashes\_cyclist).

**4. Modellansatz**  
Die Unfallrate wird mit einem Poisson-Modell geschätzt.  
Die Exposure pro Zelle wird über einen log-Offset berücksichtigt, sodass das Modell direkt die **Rate pro Fahrt** beschreibt:

$$
\log(\mu_i) = \log(\text{exposure}_i) + \beta_0 + \beta_1 \cdot \text{exposure\_decile}_i
$$

Ein negativer Wert für $\beta_1$ würde auf einen Safety-in-Numbers-Effekt hinweisen.

In [31]:
# Nur Zellen mit positiver Exposure behalten
df_h1 = df_spatial_master[df_spatial_master["exposure_total"] > 0].copy()

# Exposure in Deciles einteilen
# Crashrate entsteht indirekt durch das Modell (über Offset)
df_h1["exposure_decile"] = pd.qcut(
    df_h1["exposure_total"], q=10, labels=False
)

# Poisson-GLM mit Exposure-Offset
model_h1 = smf.glm(
    formula="crashes_cyclist ~ exposure_decile",
    data=df_h1,
    family=sm.families.Poisson(),
    offset=np.log(df_h1["exposure_total"])
).fit()

# Modellzusammenfassung ausgeben
custom_summary(model_h1)


  Kompakte Modell-Summary

Anzahl Beobachtungen: 1716
Log-Likelihood:       -3185.27
Overdispersion:       44.889
(≈1 gut, 1–3 normal, >4 auffällig)

        feature   coef exp(coef) p_value ci_2.5% ci_97.5%
      Intercept -7.517     0.001       0  -7.611   -7.423
exposure_decile -0.373     0.688       0  -0.388   -0.359


---

## 2. H2 – Zeitliche Struktur der Unfallrate (Stunde und Saison)


**1. Motivation**  
In der zeitlichen EDA zeigten sich deutliche Unterschiede in der relativen Unfallrate über den Tagesverlauf und zwischen den Jahreszeiten.  
Der Test prüft, ob diese Muster systematisch sind und ob Stunde und Saison als eigenständige Risikotreiber in Frage kommen.

**2. Formale Hypothese**  
- $H_0: \text{Die Unfallrate pro Fahrt unterscheidet sich nicht zwischen Stunden oder Saisons.}$
- $H_1: \text{Die Unfallrate pro Fahrt variiert systematisch nach Stunde und Saison.}$

**3. Datenbasis und Aggregation**  
Für jede Kombination aus Datum und Stunde wird die Exposure (Anzahl der Fahrten) bestimmt.  
Unfälle werden separat pro (date, hour) gezählt und mit Exposure verknüpft.  
Zur Kontrolle zeitlicher Muster werden zusätzlich weekday und season pro Stunde ergänzt.

**4. Modellansatz**  
Die Unfallrate wird über ein Poisson-Modell mit logarithmiertem Exposure-Offset geschätzt.  
Damit beschreibt das Modell direkt die **Rate pro Fahrt** für Stunde und Saison:

$$
\log(\mu_i) = \log(\text{exposure}_i)
+ \beta_0
+ \beta_{\text{hour}} \cdot \text{hour}_i
+ \beta_{\text{season}} \cdot \text{season}_i
+ \beta_{\text{weekday}} \cdot \text{weekday}_i
$$

Zeitliche Kategorien mit signifikanten Koeffizienten weisen auf eine systematische zeitliche Struktur in der Unfallrate hin.

In [32]:
# Exposure pro (date, hour) aggregieren
exposure_temporal = (
    df_cb_temporal
    .groupby(["date", "hour"], observed=True)
    .size()
    .reset_index(name="exposure")
)

# Unfälle pro (date, hour) aggregieren
crashes_temporal = (
    df_nypd_temporal[df_nypd_temporal["cyclist_involved"]]
    .groupby(["date", "hour"], observed=True)
    .size()
    .reset_index(name="crashes")
)

# Aggregationen mergen
df_h2 = (
    exposure_temporal
    .merge(crashes_temporal, on=["date", "hour"], how="left")
    .assign(crashes=lambda x: x["crashes"].fillna(0))
)

# Zeitfeatures für jede Stunde ergänzen
# Drop_duplicates ist sicher, da jede (date, hour)-Kombination im CitiBike-DF temporal eindeutig ist.
time_features = df_cb_temporal[["date", "hour", "weekday", "season"]].drop_duplicates()

df_h2 = df_h2.merge(time_features, on=["date", "hour"], how="left")

# Nur Zeilen mit positiver Exposure behalten
df_h2 = df_h2[df_h2["exposure"] > 0].copy()

# Poisson-GLM mit kategorialen Zeitvariablen
model_h2 = smf.glm(
    formula="crashes ~ C(hour) + C(weekday) + C(season)",
    data=df_h2,
    family=sm.families.Poisson(),
    offset=np.log(df_h2["exposure"])
).fit()

# Modellzusammenfassung ausgeben
custom_summary(model_h2)


  Kompakte Modell-Summary

Anzahl Beobachtungen: 8759
Log-Likelihood:       -7236.61
Overdispersion:       1.176
(≈1 gut, 1–3 normal, >4 auffällig)

            feature   coef exp(coef)   p_value ci_2.5% ci_97.5%
          Intercept -8.070     0.000         0  -8.255   -7.885
       C(hour)[T.1] -0.446     0.640   0.00294  -0.740   -0.152
       C(hour)[T.2] -0.494     0.610   0.00624  -0.848   -0.140
       C(hour)[T.3] -0.004     0.996     0.981  -0.348    0.339
       C(hour)[T.4] -0.115     0.891     0.547  -0.490    0.260
       C(hour)[T.5] -0.841     0.431  5.48e-06  -1.203   -0.478
       C(hour)[T.6] -1.478     0.228  3.01e-20  -1.792   -1.164
       C(hour)[T.7] -1.294     0.274  1.73e-27  -1.527   -1.060
       C(hour)[T.8] -1.462     0.232  4.96e-39  -1.681   -1.242
       C(hour)[T.9] -1.387     0.250  7.25e-34  -1.611   -1.163
      C(hour)[T.10] -1.186     0.306  6.36e-26  -1.407   -0.965
      C(hour)[T.11] -1.189     0.305  4.92e-27  -1.405   -0.972
      C(hour)[T.12

### 2.1 Ergebnisse des Poisson-Modells (H2)

Das Modell quantifiziert, ob die zeitliche Struktur des Fahrtbeginns
die Unfallrate pro Fahrt systematisch beeinflusst. Die Ergebnisse zeigen ein
deutliches und konsistentes Muster:

#### 2.1.1 Stunden-Effekte: klarer und starker Treiber
Die meisten Stundenkoeffizienten sind hochsignifikant und stark negativ 
(im Vergleich zur Referenzstunde 0 Uhr). Beispiele:

- C(hour)[T.6]  = –1.48  
- C(hour)[T.7]  = –1.29  
- C(hour)[T.8]  = –1.46  
- C(hour)[T.9]  = –1.39  

Alle mit p < 0.001.

Dies bedeutet:
**Die Crashrate pro Fahrt ist in den typischen Tagesstunden etwa 3–4× niedriger
als in der späten Nacht.**

Damit bestätigt das Modell die vorherige EDA (risk_hour-Plot) sehr robust:
- Nacht = hohes Risiko  
- Tag = deutlich niedrigeres Risiko

Der Stunden-Effekt ist der stärkste und strukturell dominierende zeitliche Treiber.

#### 2.1.2 Wochentag: schwacher Effekt
Für die meisten Wochentage sind die Effekte statistisch nicht signifikant:

- C(weekday)[T.1] → p=0.48  
- C(weekday)[T.2] → p=0.59  
- C(weekday)[T.3] → p=0.28  
- C(weekday)[T.5] → p=0.94  
- C(weekday)[T.6] → p=0.30  

Dies ist erwartbar, da im Modell **hour** bereits die dominante Variation erklärt.
Die Unterschiede zwischen Wochentagen reduzieren sich stark, wenn die 
Stundenstruktur kontrolliert wird. Das deckt sich mit der EDA:
- Risiko-Hotspots entstehen eher durch „späte Stunden“  
  als durch die Wochentage an sich.

#### 2.1.3 Saison: moderater, aber realer Effekt
- **Spring:** signifikant erhöhtes Risiko (β=+0.123, p=0.005)  
- **Winter:** grenzwertig erhöht (β=+0.093, p=0.058)  
- **Summer:** weitgehend neutral  

Interpretation:
- Saison prägt die Risikostruktur, aber weniger stark als die Stunde.
- „Frühjahr“ zeigt leicht erhöhte Crashraten (mehr Outdoor-Verkehr, 
  aber noch suboptimale Lichtverhältnisse).

#### 2.1.4 Modellgüte und Overdispersion
Pearson χ² = 1.03e+04 bei 8726 df →  
Dispersion ≈ 1.18 → **leichte Overdispersion**, aber völlig akzeptabel.

Im Vergleich zu H1 (φ ≈ 18) ist dieses Modell deutlich stabiler.
Die Effektschätzung ist robust; lediglich die Unsicherheit wäre mit NB etwas
konservativer.

#### 2.1.5 Gesamteinschätzung
Die strukturelle Aussage ist klar:

- Die Stunde des Fahrtbeginns ist ein **wesentlicher Risikotreiber**.  
- Saison zeigt **moderate, nachvollziehbare Effekte**.  
- Wochentag spielt **nur eine untergeordnete Rolle**, wenn Stunde kontrolliert wird.  
- Das Modell als Ganzes erklärt zwar nur einen kleinen Teil der Gesamtvarianz
  (Pseudo-R² ~ 0.047), aber das ist bei stündlichen Unfalldaten völlig normal.

Am wichtigsten:  
Das Muster deckt sich sauber mit der visuellen EDA, und der Effekt der Stunden 
ist stark und konsistent.

#### 2.1.6 Entscheidung über die Hypothese

**H₀:** Die Unfallrate ist unabhängig von der Fahrtbeginnzeit.  
**H₁:** Die Unfallrate variiert systematisch über Stunde, Wochentag und Saison.

Auf Basis der Modellresultate – insbesondere der starken und hochsignifikanten
Stunden-Effekte – **wird H₀ klar abgelehnt**.

**Zeit ist ein signifikanter und strukturgebender Risikotreiber.**

---

## 3. H3 – Unterschiedliche Unfallraten von E-Bikes und klassischen Fahrrädern

**1. Motivation**  
Die EDA zeigte Unterschiede im Nutzungsprofil zwischen klassischen Fahrrädern und E-Bikes.  
Um zu prüfen, ob sich diese Unterschiede auch in der Unfallrate pro Fahrt widerspiegeln, wird der Bike-Typ formal getestet – unter Kontrolle der zeitlichen Struktur (Stunde und Saison).

**2. Formale Hypothese**  
- $H_0: \text{Die Unfallrate pro Fahrt unterscheidet sich nicht zwischen klassischen Fahrrädern und E-Bikes.}$ 
- $H_1: \text{Die Unfallrate pro Fahrt unterscheidet sich systematisch zwischen den beiden Bike-Typen.}$

**3. Datenbasis und Aggregation**  
Exposure und Unfälle werden für jede Kombination aus Stunde, Saison und Bike-Typ aggregiert.  
Ambiguous-Fälle werden ausgeschlossen, da der Bike-Typ dort nicht eindeutig ist.  
Der Wochentag wird nicht einbezogen, da er im vorherigen Test keinen eigenständigen Einfluss auf die Unfallrate zeigte, sobald die Stunde modelliert wurde.

**4. Modellansatz**  
Die Unfallrate pro Fahrt wird über ein Poisson-Modell mit logarithmiertem Exposure-Offset geschätzt:

$$
\log(\mu_i) =
\log(\text{exposure}_i) +
\beta_0 +
\beta_{\text{type}} \cdot \text{rideable\_type}_i +
\beta_{\text{hour}} \cdot \text{hour}_i +
\beta_{\text{season}} \cdot \text{season}_i
$$

Es wird bewusst **nur für zeitliche Effekte** kontrolliert (hour, season), nicht jedoch für räumliche Unterschiede.  
Der Grund: Der Bike-Typ ist räumlich stark konzentriert (E-Bikes werden z. B. häufiger im Manhattan-Kern genutzt).  
Eine räumliche Kontrolle würde diesen Effekt vollständig herausrechnen und damit verhindern, zu prüfen, ob im realen Nutzungsmuster ein Unterschied zwischen den Bike-Typen sichtbar ist.

Das Modell schätzt somit die relative Unfallrate der Bike-Typen unter vergleichbaren zeitlichen Bedingungen.

In [33]:
# Exposure pro (hour, season, rideable_type)
exposure_h3 = (
    df_cb_temporal
    .groupby(["hour", "season", "rideable_type"], observed=True)
    .size()
    .reset_index(name="exposure")
)

# Unfälle mit Bike-Typ bestimmen
crash_temp = df_nypd_temporal[df_nypd_temporal["cyclist_involved"]].copy()

# Bike-Typ bestimmen (ambiguous entfernen)
crash_temp["rideable_type"] = np.where(
    crash_temp["ebike_involved"], "electric_bike",
    np.where(crash_temp["bike_involved"], "classic_bike", np.nan)
)
crash_temp = crash_temp[crash_temp["rideable_type"].notna()]

# Crashaggregation
crashes_h3 = (
    crash_temp
    .groupby(["hour", "season", "rideable_type"], observed=True)
    .size()
    .reset_index(name="crashes")
)

# Exposure und Crashes mergen
df_h3 = (
    exposure_h3
    .merge(crashes_h3,
           on=["hour", "season", "rideable_type"],
           how="left")
    .assign(crashes=lambda x: x["crashes"].fillna(0))
)

# Nur positive Exposure behalten
df_h3 = df_h3[df_h3["exposure"] > 0].copy()

# Poisson-GLM: Modell für Bike-Typ unter Kontrolle der Zeitstruktur
model_h3 = smf.glm(
    formula="crashes ~ C(rideable_type) + C(hour) + C(season)",
    data=df_h3,
    family=sm.families.Poisson(),
    offset=np.log(df_h3["exposure"])
).fit()

# Modellzusammenfassung ausgeben
custom_summary(model_h3)


  Kompakte Modell-Summary

Anzahl Beobachtungen: 192
Log-Likelihood:       -507.33
Overdispersion:       1.102
(≈1 gut, 1–3 normal, >4 auffällig)

                          feature   coef exp(coef)   p_value ci_2.5% ci_97.5%
                        Intercept -7.348     0.001         0  -7.516   -7.181
C(rideable_type)[T.electric_bike] -1.750     0.174         0  -1.837   -1.663
                     C(hour)[T.1] -0.430     0.650   0.00449  -0.727   -0.133
                     C(hour)[T.2] -0.519     0.595   0.00511  -0.882   -0.156
                     C(hour)[T.3]  0.034     1.035     0.847  -0.311    0.379
                     C(hour)[T.4] -0.047     0.954     0.807  -0.423    0.329
                     C(hour)[T.5] -0.904     0.405  1.85e-06  -1.276   -0.533
                     C(hour)[T.6] -1.547     0.213  1.11e-21  -1.864   -1.230
                     C(hour)[T.7] -1.377     0.252  1.09e-30  -1.612   -1.143
                     C(hour)[T.8] -1.631     0.196  1.61e-47  -1.852   -

---

## 4. Feature Engineering und Aufbau des finalen Modell-Datensatzes


Auf Basis der drei Hypothesentests und der vorangegangenen EDA werden nun die Features definiert, die in das spätere Risikomodell eingehen.  
Ziel ist ein kompakter, erklärbarer Datensatz, der alle relevanten Einflussgrößen enthält, ohne unnötige Komplexität oder Konfundierung einzubauen.

**Auswahl der Features**  
Die Tests haben gezeigt, dass drei Faktoren systematisch mit der Unfallrate pro Fahrt zusammenhängen:

- **Nutzungsintensität (exposure)**  
  – wird über Exposure-Deciles repräsentiert, basierend auf der gesamten Nutzung pro Zelle  
- **Zeitliche Struktur (hour, season)**  
  – Stunde und Jahreszeit sind deutliche Risikotreiber und werden direkt als kategoriale Features geführt  
- **Bike-Typ (rideable_type)**  
  – E-Bikes und klassische Fahrräder zeigen unterschiedliche Risikoprofile unter vergleichbaren zeitlichen Bedingungen  

Diese Variablen bilden zusammen die Kernstruktur, die das spätere GLM benötigt.

**Aufbau des finalen Datensatzes**  
Für jede Kombination aus Zelle (*cell_idx*), Stunde, Saison und Bike-Typ wird die entsprechende  
**Exposure** (Anzahl der Fahrten) und **Crash-Anzahl** zusammengestellt.  
Dadurch entsteht ein granularer Datensatz, der die reale Risikosituation pro Segment abbildet.

Der Datensatz enthält im Wesentlichen folgende Spalten:

- `cell_idx` – räumliche Zelle  
- `hour` – Stunde des Tages  
- `season` – Jahreszeit  
- `rideable_type` – Bike-Kategorie  
- `exposure` – Anzahl der CitiBike-Fahrten im Segment  
- `crashes` – Anzahl der Radfahrer-Unfälle im Segment  
- `exposure_decile` – Nutzungsintensität der Zelle (Aggregat über das Jahr)

Dieser Schritt verbindet damit die räumlichen, zeitlichen und nutzungsbezogenen Dimensionen zu einem konsistenten Modell-Dataset, das direkt in das finale GLM überführt werden kann.

In [34]:
# 1) Hilfsfunktion zur Ergänzung zeitlicher Features
def add_temporal_features(df, dt_col):
    df = df.copy()
    dt = df[dt_col]

    # Stunde extrahieren
    df["hour"] = dt.dt.hour.astype("int8")

    # Saison bestimmen
    month = dt.dt.month.to_numpy()
    season = np.empty_like(month)

    season[(month == 12) | (month <= 2)] = 0      # Winter
    season[(3 <= month) & (month <= 5)] = 1       # Frühling
    season[(6 <= month) & (month <= 8)] = 2       # Sommer
    season[(9 <= month) & (month <= 11)] = 3      # Herbst

    season_map = {0: "winter", 1: "spring", 2: "summer", 3: "fall"}
    df["season"] = pd.Categorical([season_map[s] for s in season])

    return df


# 2) Exposure aus CitiBike-Daten (df_cb_spatial, 35 Mio Fahrten)
# Relevante Spalten auswählen
cb = df_cb_spatial[[
    "rideable_type",
    "started_at",
    "duration_min",
    "start_cell_idx"
]].copy()

# Fahrten mit extremer Dauer entfernen
cb = cb[cb["duration_min"] <= 360]

# Zeitfeatures ergänzen
cb = add_temporal_features(cb, "started_at")

# Exposure aggregieren
exposure_df = (
    cb.groupby(["start_cell_idx", "hour", "season", "rideable_type"], observed=True)
      .size()
      .reset_index(name="exposure")
)

exposure_df = exposure_df.rename(columns={"start_cell_idx": "cell_idx"})


# 3) Crashes aggregieren (df_nypd_spatial, 61k Unfälle)
# Nur Radfahrer-Unfälle behalten
cr = df_nypd_spatial[df_nypd_spatial["cyclist_involved"]].copy()

# Ambiguous-Fälle ausschließen
cr = cr[~cr["ambiguous"]].copy()

# Zeitfeatures ergänzen
cr = add_temporal_features(cr, "crash_datetime")

# Fahrradtyp bestimmen
cr["rideable_type"] = np.where(
    cr["ebike_involved"], "electric_bike", "classic_bike"
)

# Crash aggregieren
crashes_df = (
    cr.groupby(["crash_cell_idx", "hour", "season", "rideable_type"], observed=True)
      .size()
      .reset_index(name="crashes")
)

crashes_df = crashes_df.rename(columns={"crash_cell_idx": "cell_idx"})


# 4) Exposure und Crashes zusammenführen
master = exposure_df.merge(
    crashes_df,
    on=["cell_idx", "hour", "season", "rideable_type"],
    how="left"
)

# Fehlende Crashwerte auffüllen
master["crashes"] = master["crashes"].fillna(0).astype("int32")


# 5) Exposure-Deciles pro Zelle
# Exposure total pro Zelle summieren
cell_exp = (
    master.groupby("cell_idx")["exposure"]
          .sum()
          .reset_index()
          .rename(columns={"exposure": "exposure_total"})
)

# Exposure in Deciles einteilen
cell_exp["exposure_decile"] = pd.qcut(
    cell_exp["exposure_total"],
    q=10,
    labels=False
)

# Exposure-Deciles zurückführen
master = master.merge(
    cell_exp[["cell_idx", "exposure_decile"]],
    on="cell_idx",
    how="left"
)

master["exposure_decile"] = master["exposure_decile"].astype("float32")

print(master.shape)
master.head()

(274993, 7)


,cell_idx,hour,season,rideable_type,exposure,crashes,exposure_decile
0,23,0,fall,classic_bike,1,0,2.0
1,23,0,fall,electric_bike,2,0,2.0
2,23,0,spring,classic_bike,3,0,2.0
3,23,0,summer,classic_bike,1,0,2.0
4,23,0,summer,electric_bike,3,0,2.0


In [ ]:
# Zielpfad für finale, modellfertige Aggregationen
target_path = os.path.join(DATA_MODEL, "exposure_master.parquet")

# Speichern im Parquet-Format
master.to_parquet(target_path, index=False)

print("Exposure-Master gespeichert.")
print(f"Dimensionen: {master.shape[0]:,} Zeilen × {master.shape[1]} Spalten")

Exposure-Master gespeichert.
Dimensionen: 274,993 Zeilen × 7 Spalten
